In [1]:
from torch.utils.data import DataLoader
from concept_model.dataset import CUBAttributesToClass

training_data = CUBAttributesToClass(train=True)
test_data = CUBAttributesToClass(train=False)

batch_size = 4
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X: {X.shape} {X.dtype}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X: torch.Size([4, 312]) torch.int64
Shape of y: torch.Size([4]) torch.int64


In [2]:
import torch
from torchvision.ops import MLP

model = MLP(
    in_channels=312,
    hidden_channels=[200],
    dropout=0.2,
)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X.to(torch.float))
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X.to(torch.float))
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.6f}%, Avg loss: {test_loss:>8f} \n"
    )
    return correct


Using cuda device


In [3]:
epochs = 200
max_acc = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    curr_acc = test(test_dataloader, model, loss_fn)
    if curr_acc > max_acc:
        torch.save(model.state_dict(), "attributes_to_class.pth")
        print("Saved PyTorch Model State to attributes_to_class.pth")
        max_acc = curr_acc
print("Done!")


Epoch 1
-------------------------------
loss: 5.453866  [    0/ 5994]
loss: 5.417019  [  400/ 5994]
loss: 5.247129  [  800/ 5994]
loss: 5.155419  [ 1200/ 5994]
loss: 5.255252  [ 1600/ 5994]
loss: 5.241426  [ 2000/ 5994]
loss: 5.094325  [ 2400/ 5994]
loss: 4.729543  [ 2800/ 5994]
loss: 5.064322  [ 3200/ 5994]
loss: 5.288672  [ 3600/ 5994]
loss: 5.362391  [ 4000/ 5994]
loss: 5.200106  [ 4400/ 5994]
loss: 5.213297  [ 4800/ 5994]
loss: 5.295084  [ 5200/ 5994]
loss: 4.760361  [ 5600/ 5994]
Test Error: 
 Accuracy: 13.203314%, Avg loss: 4.931679 

Saved PyTorch Model State to attributes_to_class.pth
Epoch 2
-------------------------------
loss: 5.179961  [    0/ 5994]
loss: 4.719658  [  400/ 5994]
loss: 4.796978  [  800/ 5994]
loss: 5.125695  [ 1200/ 5994]
loss: 4.592096  [ 1600/ 5994]
loss: 4.848586  [ 2000/ 5994]
loss: 4.620932  [ 2400/ 5994]
loss: 4.591046  [ 2800/ 5994]
loss: 5.014524  [ 3200/ 5994]
loss: 4.304016  [ 3600/ 5994]
loss: 4.977604  [ 4000/ 5994]
loss: 4.175273  [ 4400/ 5994]
